<a href="https://colab.research.google.com/github/mohammad-rahbari/federated-learning_visual-classification/blob/xin_branch/notebooks/federated_model_visual_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing DINO and installing its dependencies

In [ ]:
# @title Clon the DINO ripo
!git clone https://github.com/facebookresearch/dino.git

Cloning into 'dino'...
remote: Enumerating objects: 175, done.
remote: Total 175 (delta 0), reused 0 (delta 0), pack-reused 175 (from 1)
Receiving objects: 100% (175/175), 24.47 MiB | 29.37 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [ ]:
# @title Installing required dependencies regarding DINO
%cd dino
!pip install -r requirements.txt
!pip install timm

/content/dino
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjit


# preprocessing the CIFAR-100 dataset

feature size in CIFAR is 32x32 but DINO requires 224x224 in the input layer.

In first step we upscale the dataset and then we add randomization to it

In last step of transformation we normalize data usind mean value and standard division of ImageNet



In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split,DataLoader

In [ ]:
# transform_train = transforms.Compose([
#     transforms.Resize(256),
#     transforms.RandomCrop(224),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=(0.485, 0.456, 0.406),
#                          std=(0.229, 0.224, 0.225))
# ])

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))
])





In [ ]:


full_train = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)





100%|██████████| 169M/169M [00:03<00:00, 48.6MB/s]


# Set Hyperparameters regarding the data spliting here!

In [ ]:
#@title set the parameters here!!

number_of_clients = None
train_frac = 0.8 #@param
val_frac = 0.2 #@param
is_seed_fixed = True #@param{type:"boolean"}
seed = 42 #@param{type:"integer"}




#@markdown </br> <h5>Indicate the number of clients that contribute in training:</h5>
n_clients = 10 #@param{type:"integer"}

#@markdown </br></br> <b>splitting hyperparameters</b>

spliting_method = "non-i.i.d. sharing" #@param["i.i.d. sharing","non-i.i.d. sharing"]




In [ ]:
#@title Set the parameters here only if <b>non-i.i.d. sharing</b> method had been selected!!
#@markdown Nc is the number of classes that each subset can contain
if spliting_method == "non-i.i.d. sharing":
  Nc = 10 #@param{type:"integer"}

  # are_classes_overlaping = False #@param{type:"boolean"}

#@markdown <h3>If we consider the Number of classes M and nummber of client K then:</h3>
#@markdown <ul>
#@markdown   <li>Nc should be:
#@markdown     <ul>
#@markdown       <li>
#@markdown         Greater than or equal to <b>\\(\frac{M}{K}\\)</b>
#@markdown       </li>
#@markdown       <li>
#@markdown         Less than or equal to K </b>
#@markdown       </li>
#@markdown     </ul>
#@markdown   </li>
#@markdown   <li>
#@markdown   Muximum number of clients means all classes contribute in every client
#@markdown   </li>

#@markdown </ul>


#@markdown </br></br><h3>Combination of classes are randomly selected which suits definition of federated learning especially Cross-device federated learning</h3>





# Data splitting

In [ ]:
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset
import numpy as np

generator = torch.Generator().manual_seed(seed)

total_size = len(full_train)
train_size = int(train_frac * total_size)
val_size   = total_size - train_size

train_dataset, val_dataset = random_split(full_train, [train_size, val_size], generator=generator)



print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

lenghts = [train_size//n_clients] * n_clients

for i in range(train_size % n_clients):
  lenghts[i] += 1
print("Size of subset: ", lenghts)






# if spliting_method == "i.i.d. sharing":


Train dataset size: 40000
Validation dataset size: 10000
Size of subset:  [1740, 1740, 1740, 1739, 1739, 1739, 1739, 1739, 1739, 1739, 1739, 1739, 1739, 1739, 1739, 1739, 1739, 1739, 1739, 1739, 1739, 1739, 1739]
